In [1]:
UnitOrVector{T} = Union{T, Vector{T}} where T

Union{Array{T,1}, T} where T

In [2]:
mutable struct Foo
    a::Int64
    b::UnitOrVector{Int64}
    c::Float64
    d::UnitOrVector{Float64}
    
    function Foo()
        new(0,0,0.0,0.0)
    end

end

In [3]:
for n in fieldnames(Foo)
    println(fieldtype(Foo, n))
end

Int64
Union{Int64, Array{Int64,1}}
Float64
Union{Float64, Array{Float64,1}}


In [4]:
macro grid(T)
    args = @eval fieldnames($(T))
    typename = @eval $(T)
    quote
        function $(esc(T))($(args...))
            indices = CartesianIndices(Tuple(length(i) for i in Any[$(args...)]))
            results = Vector{$(esc(T))}()
            for ci in indices
                obj = $(esc(T))()
                for (i,j) in enumerate(ci.I)
                    setfield!(obj, $(args)[i], Any[$(args...)][i][j])
                end
                push!(results, obj)
            end
            return results
        end
    end
end

@grid (macro with 1 method)

In [49]:
macro makegrid(T, args...)
    quote
        argslist = Any[$(args...)]
        fieldslist = fieldnames($T)
        indices = CartesianIndices(Tuple(length(i) for i in argslist))
        results = Vector{$T}()
        for ci in indices
            obj = $T()
            for (i,j) in enumerate(ci.I)
                setfield!(obj, fieldslist[i], argslist[i][j])
            end
            push!(results, obj)
        end
        results
    end
end

@makegrid (macro with 1 method)

In [50]:
a = 3.0
@makegrid(Foo, 1:3, 2, a, 4.0:0.5:6.0)

15-element Array{Foo,1}:
 Foo(1, 2, 3.0, 4.0)
 Foo(2, 2, 3.0, 4.0)
 Foo(3, 2, 3.0, 4.0)
 Foo(1, 2, 3.0, 4.5)
 Foo(2, 2, 3.0, 4.5)
 Foo(3, 2, 3.0, 4.5)
 Foo(1, 2, 3.0, 5.0)
 Foo(2, 2, 3.0, 5.0)
 Foo(3, 2, 3.0, 5.0)
 Foo(1, 2, 3.0, 5.5)
 Foo(2, 2, 3.0, 5.5)
 Foo(3, 2, 3.0, 5.5)
 Foo(1, 2, 3.0, 6.0)
 Foo(2, 2, 3.0, 6.0)
 Foo(3, 2, 3.0, 6.0)

In [61]:
@grid Foo

Foo

In [11]:
@macroexpand @grid Foo

quote
    #= In[9]:5 =#
    function Foo(#31#a, #32#b, #33#c, #34#d)
        #= In[9]:6 =#
        #25#indices = (Main.CartesianIndices)((Main.Tuple)(((Main.length)(#29#i) for #29#i = (Main.Any)[#31#a, #32#b, #33#c, #34#d])))
        #= In[9]:7 =#
        #26#results = (Main.Vector){Foo}()
        #= In[9]:8 =#
        for #27#ci = #25#indices
            #= In[9]:9 =#
            #28#obj = Foo()
            #= In[9]:10 =#
            for (#29#i, #30#j) = (Main.enumerate)((#27#ci).I)
                #= In[9]:11 =#
                (Main.setfield!)(#28#obj, ((:a, :b, :c, :d))[#29#i], (((Main.Any)[#31#a, #32#b, #33#c, #34#d])[#29#i])[#30#j])
            end
            #= In[9]:13 =#
            (Main.push!)(#26#results, #28#obj)
        end
        #= In[9]:15 =#
        return #26#results
    end
end

In [8]:
Foo(1, 2, 3.0, 4.0)

1-element Array{Foo,1}:
 Foo(1, 2, 3.0, 4.0)

In [6]:
Foo(1, [2,3,4], 3.14, [1.0, 2.9, 7.8])

9-element Array{Foo,1}:
 Foo(1, 2, 3.14, 1.0)
 Foo(1, 3, 3.14, 1.0)
 Foo(1, 4, 3.14, 1.0)
 Foo(1, 2, 3.14, 2.9)
 Foo(1, 3, 3.14, 2.9)
 Foo(1, 4, 3.14, 2.9)
 Foo(1, 2, 3.14, 7.8)
 Foo(1, 3, 3.14, 7.8)
 Foo(1, 4, 3.14, 7.8)

In [ ]:
args = [[1,2,3], 2.4, [4,5,6], [7,8], [true, false]]
eval(quote
    Array{NTuple{$length(args), Any}, $length(args)}(undef, $([length(a) for a in args]...))
end)[:]
for ci in CartesianIndices(Tuple(length(i) for i in args))
    println(ci, [args[i][j] for (i,j) in enumerate(ci.I)])
end